In [ ]:
import os
import uvicorn
import asyncio
from typing import Union
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
import uuid
from PIL import Image
import numpy as np
from keras.models import load_model as keras_load_model

IMAGEDIR = "images/"
if not os.path.exists(IMAGEDIR):
    os.makedirs(IMAGEDIR)

app = FastAPI()

model_path = "C:\\Users\\Dell\\Downloads\\mnist-epoch-10.hdf5"
model = None

def load_model(model_path):
    return keras_load_model(model_path)

async def get_model():
    global model
    if model is None:
        model = load_model(model_path)
    return model

def format_image(image):
    # Convert the image to grayscale and resize to 28x28
    image = image.convert("L").resize((28, 28))
    # Convert the image to a numpy array
    image_array = np.array(image)
    # Flatten and normalize the pixel values
    normalized_image = image_array.reshape(1, 784) / 255.0
    return normalized_image

async def predict(img):
    model = await get_model()
    x = format_image(img)
    prediction = model.predict(x)
    digit = np.argmax(prediction)
    return str(digit)

@app.post("/upload/")
async def create_upload_file(file: UploadFile = File(...)):
    file.filename = f"{uuid.uuid4()}.jpg"
    contents = await file.read()
    # Save the file
    with open(f"{IMAGEDIR}{file.filename}", "wb") as f:
        f.write(contents)

    img = Image.open(f'{IMAGEDIR}{file.filename}')
    ans = await predict(img)

    os.remove(f'{IMAGEDIR}{file.filename}')
    return {"digit": ans}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
